# Data Forensics Project Team 10 - Suze Sips & Nadine van Raaij

## Configure Telegram API
Based on the Telethon library from https://pypi.org/project/Telethon/ and https://betterprogramming.pub/how-to-get-data-from-telegram-82af55268a4b

In [58]:
# Imports
import configparser
import json
import csv

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (PeerChannel)

import asyncio
from telethon.sync import TelegramClient
from telethon import TelegramClient, events
from telethon.errors.rpcerrorlist import ChannelPrivateError
from telethon.errors.rpcerrorlist import FloodWaitError

In [37]:
# Create a Telegram clients -> this is in the config.ini file
# Example is given, no need for quotes

# You can get your telegram development credentials in telegram API Development Tools
# api_id = 12345678
# api_hash = 123a12abc123456a1a1abcd12345abcd

# Use full phone number including + and country code
# phone = +31612345678
# username = @example_name

In [2]:
# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

In [3]:
# Create the client (object) and connect
client = TelegramClient(username, api_id, api_hash)
client.start()
print("Client Created")
# Ensure you're authorized
if not client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

Client Created


C:\Users\20184116\AppData\Local\Temp\ipykernel_17140\3267974630.py:3: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  client.start()
C:\Users\20184116\AppData\Local\Temp\ipykernel_17140\3267974630.py:6: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():


In [14]:
# Request for channel members from Telegram

user_input_channel = input("enter entity(telegram URL or entity id):")

if user_input_channel.isdigit():
    entity = PeerChannel(int(user_input_channel))
else:
    entity = user_input_channel

my_channel = client.get_entity(entity)

enter entity(telegram URL or entity id):https://t.me/WeTheMedia


## Get Participant details
It was found that for the crawled channel, the participants details could not be crawled as they were protected by the channel owner. However, this code was tested on a private channel that was created. This channel included only the telegram user account which was used to crawl the data. Therefore the .JSON and .CSV files are empty in this project. 

In [40]:
# Function to list all the participants of the channel with a maximum set to 100
async def get_all_participants(client, my_channel):
    all_participants = []
    offset_id = 0
    limit = 100
    while True:
        participants = await client(GetParticipantsRequest(
            my_channel, ChannelParticipantsSearch(''), offset_id, limit, hash=0
        ))
        if not participants.users:
            break
        all_participants.extend(participants.users)
        offset_id = participants.users[-1].id
    return all_participants

# Example usage -> code showed after running it for the second time an error that database is locked as the session ended
# A new session should be opened to run the function again

async def main():
    # Initialize the Telegram client
    client = TelegramClient('session_name', api_id, api_hash)
    await client.start()

    # Get all participants
    my_channel = my_channel
    all_participants = await get_all_participants(client, my_channel)

    # Print the participants
    for participant in all_participants:
        print(participant)

    # Disconnect the client
    await client.disconnect()

asyncio.run(main())


OperationalError: database is locked

In [41]:
# Dump into a .JSON file
all_user_details = []
for participant in all_participants:
    all_user_details.append(
        {"id": participant.id, "first_name": participant.first_name, "last_name": participant.last_name,
         "user": participant.username, "phone": participant.phone, "is_bot": participant.bot})

with open('user_data.json', 'w') as outfile:
    json.dump(all_user_details, outfile)
    

In [52]:
# Add user information to the .CSV file
def save_users_to_csv(client, chat_id):
    # Retrieve all participants in the chat
    all_participants = client.get_participants(chat_id)
    
    # Save details of all users to a .CSV file
    with open('user_data.csv', mode='w', newline='') as outfile:
        fieldnames = ['id', 'first_name', 'last_name', 'username', 'phone', 'is_bot']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for participant in all_participants:
            writer.writerow({'id': participant.id, 'first_name': participant.first_name,
                             'last_name': participant.last_name, 'username': participant.username,
                             'phone': participant.phone, 'is_bot': participant.bot})


## Get Messages sent in channel

In [44]:
# Get channel messages without filter of word_list with a maximum of 100 words

# offset_id = 0
# limit = 100
# all_messages = []
# total_messages = 0
# total_count_limit = 0

# while True:
#     print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
#     history = client(GetHistoryRequest(
#         peer=my_channel, # Channel object
#         offset_id=offset_id,
#         offset_date=None,
#         add_offset=0,
#         limit=limit,
#         max_id=0,
#         min_id=0,
#         hash=0
#     ))
#     if not history.messages:
#         break
#     messages = history.messages
#     for message in messages:
#         all_messages.append(message.to_dict())
#     offset_id = messages[len(messages) - 1].id
#     total_messages = len(all_messages)
#     if total_count_limit != 0 and total_messages >= total_count_limit: # if you want all set to 0
#         break

Task exception was never retrieved
future: <Task finished name='Task-834' coro=<TelegramBaseClient._disconnect_coro() done, defined at C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\client\telegrambaseclient.py:657> exception=OperationalError('database is locked')>
Traceback (most recent call last):
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\asyncio\tasks.py", line 232, in __step
    result = coro.send(None)
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\client\telegrambaseclient.py", line 691, in _disconnect_coro
    self.session.process_entities(types.contacts.ResolvedPeer(None, [e._as_input_peer() for e in entities], []))
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\sessions\sqlite.py", line 301, in process_entities
    c.executemany(
sqlite3.OperationalError: database is locked


## Get messages containing indicator words - CTER specific crawler

In [42]:
# Crawl only messages that contain at least one word of the words_to_find list
async def crawl_messages(api_id, api_hash, phone_number, channel_username, words_to_find):
    # Create a new TelegramClient
    client = TelegramClient('session_name', api_id, api_hash)

    # Connect to the Telegram API
    await client.connect()

    # Authenticate with the Telegram API
    if not await client.is_user_authorized():
        await client.send_code_request(phone_number)
        await client.sign_in(phone_number, input('Enter the code: '))

    # Get the channel entity
    try:
        entity = await client.get_entity(channel_username)
    except ChannelPrivateError:
        print("Error: Channel is private")
        return

    # Define a function to check if a message contains any of the words to find
    def message_contains_word(message, words):
        for word in words:
            if word.lower() in message.message.lower():
                return True
        return False

    # Define an event handler for new messages
    @client.on(events.NewMessage(chats=[entity]))
    async def handle_new_message(event):
        if message_contains_word(event.message, words_to_find):
            print(event.message.text)

    # Start the event loop
        # Start the event loop with a timeout of 10 seconds
    try:
        await asyncio.wait_for(client.run_until_disconnected(), timeout=10.0)
    except asyncio.TimeoutError:
        pass
#     await client.run_until_disconnected()
#     client.disconnect()

In [43]:
# Create list of indicator words from the .CSV file
def read_csv_file(file_path):
    data = []
    with open(file_path, 'r',) as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)
    return data

# Specify the path to your CSV file
csv_file_path = '/word_list.csv'

# Read the CSV file and store the contents in a list
word_list = read_csv_file(csv_file_path)

# Print the list of data
for row in csv_data:
    print(row)


['brotherhood']
['Coercive']
['Permissive Societies']
['radicalization']
['terror']
['extremist']
['criminal']
['insurgent']
['terror']
['terrorism']
['fear']
['violence']
['militant']
['violent']
['state terrorism']
['attacks']
['radical']
['horrific']
['horror']
['separatist']
['terrible']
['suicide']
['freedom']
['paramilitary']
['jihad']
['military']
['horrendous']
['horrible']
['bombers']
['vigilante']
['rebel']
['hate']
['civilian']
['murderer']
['bin laden']
['usama bin laden']
['fedayeen']
['war']
['political']
['militia']
['mule']
['suicide bomber']
['shooter']
['islamic']
['cult']
['al-qaeda']
['fighter']
['anti-terrorism legislation']
['war crime']
['racist']
['left-wing politics']
['mass']
['israel']
['revolutionary']
['kidnapping']
['guerilla']
['cartel']
['jacobin club']
['fascist']
['traitor']
['terrorists']
['reign of terror']
['freedom fighter']
['shooting']
['communist']
['resistance']
['insurrection']
['united nations']
['crime']
['gang']
['timothy mcveigh']
['bruce 

Fatal error handling updates (this is a bug in Telethon, please report it)
Traceback (most recent call last):
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\client\updates.py", line 287, in _update_loop
    diff = await self(get_diff)
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\client\users.py", line 30, in __call__
    return await self._call(self._sender, request, ordered=ordered)
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\client\users.py", line 85, in _call
    self.session.process_entities(result)
  File "C:\Users\20184116\Anaconda3\envs\v_env\lib\site-packages\telethon\sessions\sqlite.py", line 301, in process_entities
    c.executemany(
sqlite3.OperationalError: database is locked


In [45]:
# Example usage -> code showed after running it for the second time an error that database is locked as the session ended
# A new session should be opened to run the function again

await crawl_messages(api_id, api_hash, phone, username, word_list)

OperationalError: database is locked

In [57]:
# Dump the crawled messages in a .CSV file
async def crawl_and_filter_telegram_messages(api_id, api_hash, chat_url, words_to_filter):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        messages = await client.get_messages(chat_url)
        filtered_messages = []
        for message in messages:
            if any(word.lower() in message.message.lower() for word in words_to_filter):
                filtered_messages.append({"id": message.id, "date": message.date.isoformat(), "text": message.message})
        return filtered_messages

async def dump_telegram_messages_to_csv(messages, filename):
    fieldnames = ["id", "date", "text"]
    with open(filename, mode='w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for message in messages:
            writer.writerow(message)

async def main():
    api_id = 12345678 # same as in config.ini file 
    api_hash = "123a12abc123456a1a1abcd12345abcd" # same as in config.ini file
    chat_url = "https://t.me/WeTheMedia"
    words_to_filter = word_list
    filtered_messages = await crawl_and_filter_telegram_messages(api_id, api_hash, chat_url, words_to_filter)
    await dump_telegram_messages_to_csv(filtered_messages, 'filtered_messages.csv')

if __name__ == '__main__':
    asyncio.run(main())


In [37]:
chat_url = "https://t.me/WeTheMedia"
filtered_messages = crawl_and_filter_telegram_messages(api_id, api_hash, chat_url, word_list)
dump_telegram_messages_to_csv(filtered_messages, "filtered_messages.csv")

<coroutine object dump_telegram_messages_to_csv at 0x0000016485780F20>